In [1]:
 
import glob
from medpy.io import load 

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from sklearn.preprocessing import MinMaxScaler
 

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [2]:
n_image = 25

## Read Files

In [3]:
dir_path = "./brats2015/BRATS2015_Training/BRATS2015_Training/HGG"

In [4]:
files_list = glob.glob(dir_path + "/*/*/*.mha")

In [18]:
dir_path_test = "./brats2015/BRATS2015_Testing/Testing/HGG_LGG"

In [19]:
files_list = glob.glob(dir_path_test + "/*/*/*.mha")

In [20]:
files_list[:5]

['./brats2015/BRATS2015_Testing/Testing/HGG_LGG\\brats_2013_pat0103_1\\VSD.Brain.XX.O.MR_Flair.54193\\VSD.Brain.XX.O.MR_Flair.54193.mha',
 './brats2015/BRATS2015_Testing/Testing/HGG_LGG\\brats_2013_pat0103_1\\VSD.Brain.XX.O.MR_T1.54194\\VSD.Brain.XX.O.MR_T1.54194.mha',
 './brats2015/BRATS2015_Testing/Testing/HGG_LGG\\brats_2013_pat0103_1\\VSD.Brain.XX.O.MR_T1c.54195\\VSD.Brain.XX.O.MR_T1c.54195.mha',
 './brats2015/BRATS2015_Testing/Testing/HGG_LGG\\brats_2013_pat0103_1\\VSD.Brain.XX.O.MR_T2.54196\\VSD.Brain.XX.O.MR_T2.54196.mha',
 './brats2015/BRATS2015_Testing/Testing/HGG_LGG\\brats_2013_pat0105_1\\VSD.Brain.XX.O.MR_Flair.54199\\VSD.Brain.XX.O.MR_Flair.54199.mha']

In [21]:
image_data, image_header = load(files_list[0])

In [22]:
image_data.shape

(240, 240, 155)

Create dataset by selecting the MR image of type FLAIRE, T2, and OT Segementation. 

to reduce the dataset, select 60 images randomly in range [50, 110]. 

the deepth is noted in the file name. 

In [23]:
files_T2 = [f for f in files_list if "T2" in f ]

In [24]:
len(files_T2)

110

In [25]:
files_OT = [f for f in files_list if "OT" in f ]

In [26]:
len(files_OT)

0

In [27]:
files_Flair = [f for f in files_list if "Flair" in f ]

In [28]:
len(files_Flair)

110

## Creating images dataset

In [14]:
#the start and end range of selection
start_idx, end_idx, step = 30, 120, 5

#prefixed name of folders storing iamges
prefixed_name_T2 = "./BraTS2015_Medium/T2/HGG_"

prefixed_name_OT = "./BraTS2015_Medium/OT/HGG_"

prefixed_name_OT_BACKGROUND = "./BraTS2015_Medium/OT_BACKGROUND/HGG_"

prefixed_name_OT_BACKGROUND_test = "./BraTS2015_Medium/OT_BACKGROUND/test/HGG_"

In [15]:
import os
import errno
try:
    os.makedirs(os.path.dirname('./BraTS2015_Medium/T2/'))
except OSError as exc: # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise
        
try:
    os.makedirs(os.path.dirname('./BraTS2015_Medium/OT/'))
except OSError as exc: # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise
        
try:
    os.makedirs(os.path.dirname('./BraTS2015_Medium/OT_BACKGROUND/'))
except OSError as exc: # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise
        

In [16]:
#scale image and save it
for i in range(len(files_T2)):
    image_data, image_header = load(files_T2[i])
    img_min, img_max = image_data.min(), image_data.max()
    scaled_image_data = (image_data - img_min) / img_max
    for j in range(start_idx, end_idx, step): 
        save_image(torch.tensor(scaled_image_data[:, :, j], dtype=torch.float32), prefixed_name_T2 + ("%03d" % i) + "_" + ("%03d" % j)+'.png')

In [17]:
#scale image and save it
for i in range(len(files_OT)):
    image_data, image_header = load(files_OT[i])
    img_min, img_max = image_data.min(), image_data.max()
    scaled_image_data = (image_data - img_min) / img_max
    for j in range(start_idx, end_idx, step): 
        save_image(torch.tensor(scaled_image_data[:, :, j], dtype=torch.float32), prefixed_name_OT + ("%03d" % i) + "_" + ("%03d" % j)+'.png')

In [30]:
#scale image and save it ther version of OT with background 
for i in range(len(files_OT)):
    
    image_data_back, image_header_back = load(files_T2[i]) 
    img_min, img_max = image_data_back.min(), image_data_back.max()
    scaled_image_data_back = (image_data_back - img_min) / img_max
    scaled_image_data_back[scaled_image_data_back > 0] = 0.2
    
    image_data, image_header = load(files_OT[i])
    img_min, img_max = image_data.min(), image_data.max()
    scaled_image_data = (image_data - img_min) / img_max
    scaled_image_data += scaled_image_data_back
    
    for j in range(start_idx, end_idx, step): 
        save_image(torch.tensor(scaled_image_data[:, :, j], dtype=torch.float32), prefixed_name_OT_BACKGROUND + ("%03d" % i) + "_" + ("%03d" % j)+'.png')